In [ ]:
import gpmultipy, scipy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from gpmultipy import dataset
from gpmultipy.model import Model
from gpmultipy.prior import Prior
from gpmultipy.kernel import RBF, White, Addition
from gpmultipy.freeze import Freezer
from gpmultipy.sampler.slice import Slice
from gpmultipy.kernel import Product, Linear

In [ ]:
%matplotlib inline

In [ ]:
ds = dataset.DataSet('data/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',effects=['Well'])
y = np.log2(y)
y = (y-y.mean())/y.std()

In [ ]:
edge = range(101,111) + range(111,191,10) + range(120,191,10) + range(191,201) + range(201,211) + range(211,291,10) + range(220,291,10) + range(291,301)
edge = np.array(edge)
edge

In [ ]:
plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.4);

In [ ]:
dist = np.array([min(abs(l-edge)) for l in labels[0]])
position = (dist==0).astype(int)

In [ ]:
[plt.plot(x,y[:,i],alpha=.4,color=c) for i,c in zip(range(y.shape[1]), ['C0' if p==0 else 'C2' for p in position])];

plt.xlabel("time", fontsize=15)
plt.ylabel("log(OD)", fontsize=15)

plt.savefig("figures/edge-effect-data-single.pdf")

# plt.plot(x,y[:,position==0].mean(1),c='r',alpha=1)
# plt.plot(x,y[:,position==1].mean(1),c='g',alpha=1)

In [ ]:
plt.figure(figsize=(10,6))
for i,p in enumerate([0,1]):
    select = position==p
    
    plt.subplot(2,2,i+1)
    plt.plot(x,y[:,select],c='k',alpha=.4)
    plt.plot(x,y[:,select].mean(1),c='r',alpha=1)
    plt.ylim(y.min(),y.max())
    
    plt.subplot(2,2,i+3)
    plt.plot(x,y[:,select]-y[:,select].mean(1)[:,None],c='k',alpha=.4)
    plt.ylim(-.6,.6)
    
plt.subplot(223)
plt.xlabel("time", fontsize=15)
plt.ylabel("log(OD)", fontsize=15)
plt.subplot(224)
plt.xlabel("time", fontsize=15)

plt.tight_layout()

plt.savefig("figures/edge-effect-data.pdf")

In [ ]:
plt.plot(x,y[:,position==0].mean(1) - y[:,position==1].mean(1))

In [ ]:
def wellPosition(number):
    number = int(number)
    if number > 100:
        return wellPosition(number%100)
    if number == 0:
        return 9,9
    return [(number-1) % 10, (number-1)/10]

# zip(labels[0],map(wellPosition,labels[0]))

In [ ]:
position = np.array(map(wellPosition,labels[0]))

In [ ]:
dist = np.column_stack((position.min(1),(9-position).min(1))).min(1)
dist

In [ ]:
plt.figure(figsize=(12,8))
for i,d in enumerate(np.unique(dist)):
    select = dist==d
    
    plt.subplot(2,3,i+1)
    plt.title(d)
    plt.plot(x,y[:,select],c='C%d'%i,alpha=.4)
    plt.ylim(y.min(), y.max())

plt.subplot(231)
plt.ylabel("log(OD)",fontsize=15)
plt.subplot(2,3,4)
plt.xlabel("time (h)",fontsize=15)
plt.ylabel("log(OD)",fontsize=15)
plt.subplot(2,3,5)
plt.xlabel("time (h)",fontsize=15)

plt.savefig("figures/edge-effect-data-distance.pdf", bbox_inches='tight')